In [2]:
from load import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
d = Dataset("cellcycle", nan_strategy="none")

In [7]:
from sklearn.pipeline import Pipeline
from hiclass import MultiLabelLocalClassifierPerNode
from hiclass.metrics import make_leveled, precision, recall, f1
from sklearn.tree import DecisionTreeClassifier
from feature_selection import ModSelectKBest
from handle_nan import NumericImputer

pipeline = Pipeline([
    ("imputer", NumericImputer(strategy="mean")),
    ("selector", ModSelectKBest(k=10)),
    ("model", MultiLabelLocalClassifierPerNode(DecisionTreeClassifier()))
])

In [34]:
pipeline.fit(d.x_train(expand=True), d.y_train(expand=True))
y_pred = pipeline.predict(d.x_test())

c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ModSelectKBest was fitted without feature names
  warnings.warn(
c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but ModSelectKBest was fitted without feature names
  warnings.warn(


In [19]:
lev_y_pred = make_leveled(y_pred)

In [35]:
y_train = d.y_train(expand=False)
y_train

0          [[11, 02, 01], [11, 02, 02], [11, 02, 03, 01]]
1                                          [[12, 04, 02]]
2       [[01, 04], [14, 01], [14, 04], [16, 01], [16, ...
3              [[10, 03, 02], [42, 10], [43, 01, 03, 09]]
4                            [[10, 01, 05, 01], [32, 01]]
                              ...                        
1623                                   [[11, 02, 03, 04]]
1624    [[16, 19, 03], [20, 01, 27], [20, 03, 22], [32...
1625                 [[11, 02, 03, 01], [11, 02, 03, 04]]
1626                 [[32, 05, 01, 03, 03], [32, 07, 03]]
1627                               [[20, 01, 01, 01, 01]]
Name: class, Length: 1628, dtype: object

In [56]:
y_true = d.y_test()
f1(fill_reshape(y_true), y_pred, average="macro")

ZeroDivisionError: float division by zero

In [55]:
print(fill_reshape(y_true).shape)
print(y_pred.shape)

(1281, 13, 6)
(1281, 14, 2)


c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:75: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_len = y.apply(len).agg(max)
c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:76: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  depth = y.apply(lambda x: max([len(label) for label in x])).agg(max)


In [36]:
from sklearn.tree import DecisionTreeClassifier

from hiclass.MultiLabelLocalClassifierPerNode import MultiLabelLocalClassifierPerNode

# Define data
X_train = [[1, 2], [3, 4], [5, 6]]
X_test = [[1, 2], [3, 4], [5, 6]]

# Define Labels
Y_train = np.array([
    [["Retriever", "Golden Retriever"], ["Hound", "Dachshund"]],
    [["Retriever", "Labrador"]],
    [["Hound", "Dachshund"], ["Hound", "Beagle"]],
], dtype=object)

# Use decision tree classifiers for every node
tree = DecisionTreeClassifier()
classifier = MultiLabelLocalClassifierPerNode(local_classifier=tree)

# Train local classifier per node
classifier.fit(X_train, Y_train)

# Predict
predictions = classifier.predict(X_test)
print(predictions)

[[['Retriever' 'Golden Retriever']
  ['Hound' 'Dachshund']]

 [['Retriever' 'Labrador']
  ['' '']]

 [['Hound' 'Dachshund']
  ['Hound' 'Beagle']]]


In [49]:
from feature_selection import fill_reshape

f1(fill_reshape(pd.Series(Y_train)), predictions, average="macro")

c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:75: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  max_len = y.apply(len).agg(max)
c:\Users\sjtom\Documents\MUNI\Semester jar 2024\PV056 - Machine Learning and Data Mining\hiclass\feature_selection.py:76: FutureWarning: The provided callable <built-in function max> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  depth = y.apply(lambda x: max([len(label) for label in x])).agg(max)


1.0